<a href="https://colab.research.google.com/github/mpustil/DoctoradoPustilnik/blob/main/Modelos_SwiftV06_RF_ranking_SBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Modelo 06: ranking SBS  (1/12/2025)
# preprocesamieto - imputacion lineal

import pandas as pd
import numpy as np
import gc

'''
Archivos utilizados
Swift05_11SBS165.csv
https://drive.google.com/file/d/1T53fIqyrr70CHqXvB5lp0deJTxrfHUfV/view?usp=sharing
Swift05_11SBS52.csv
https://drive.google.com/file/d/1dQ5puACEkvwEE3zOl1vIrPglVQHXO7gD/view?usp=sharing

Swift11NoSBS.csv (solo registros sin flags de error)
https://drive.google.com/file/d/1-kSWbAy4ZL73eb6Wik3cei1s4iA1H1-j/view?usp=drive_link

'''
# To obtain a deterministic behaviour during subsampling and fitting, random_state has to be fixed to an integer
random_state = 41

ruta = "/content/drive/MyDrive/Tesis Doctorado Pustilnik/Swift/"

#preprocesamiento y union de los dataset. Variable target: SBS sii df['52']==1
def prepM3(archivo, clase, archivo2 = ''):
  df = pd.read_csv(archivo, delimiter = ";")

  if archivo2 != '':
    df2 = pd.read_csv(archivo2, delimiter = ";")
    df = pd.concat([df,df2])
    del df2
    gc.collect()


  cols = ['2', '4']
  df[cols] = df[cols].applymap(np.int64)
  #variable target
  df['52'] = clase

  df.drop(['2', '112', '113', '114', '115'], axis=1, inplace=True)
  if clase == 1:
    #df.drop(['50','51','53'], axis=1, inplace=True)
    #Nos quedamos con el nombre
    df.drop(['50','51'], axis=1, inplace=True)
  else:
    df['53'] = ''

  # ponemos los NA
  listaVariables = ['110','111']
  df[listaVariables] = df[listaVariables].replace(0, np.nan)

  #que tenga al menos uno de los filtros
  df = df[(df['110'] >= 0) | (df['111'] >=0 ) ]
  if clase == 0:
  #que no tenga flags
     df = df[(df['20'] == 0) & (df['21'] ==0 ) ]
     df.drop(['20','21'], axis=1, inplace=True)

  return df



#-------------------------------------------------------------imputacion lineal (en datos sin resumir)
from sklearn.linear_model import LinearRegression
import math

#y = ax+b  1 variable por ahora
def imputarLineal(x ,y, a, b):
    if math.isnan(y):
      ret = a*x+b
    else:
      ret = y
    return ret

#Imputacion por regresion lineal
def regresionLineal(df):
  # Dropping the entire rows with missing values
  df_drop = df.dropna()
  #print("total con 110 y 111 ",len(df_drop))
  #print(df_drop)

  # Fitting the linear regression model
  X = df_drop[['110']]
  y = df_drop['111']
  model110 = LinearRegression().fit(X, y)

  X = df_drop[['111']]
  y = df_drop['110']
  model111 = LinearRegression().fit(X, y)

  #print(f"Coefficients: {model110.coef_[0]}, Intercept: {model110.intercept_}")
  #print(f"Coefficients: {model111.coef_[0]}, Intercept: {model111.intercept_}")

  #print()
  #print(df.corr())
  df['111'] = df.apply(lambda x: imputarLineal(x['110'], x['111'], model110.coef_[0], model110.intercept_), axis=1)
  df['110'] = df.apply(lambda x: imputarLineal(x['111'], x['110'], model111.coef_[0], model111.intercept_), axis=1)



#-------------------------- Pre-procesamiento parte 2: submuestreo y sobremuestreo (codigo). ya viene todo imputado!

# importamos librerias para balancear los datos
from imblearn.under_sampling import RandomUnderSampler

#SMOTE (Synthetic Minority Over-sampling Technique)
#https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html
from imblearn.over_sampling import SMOTE

def subMuestrear(df, _random_state):

  # vamos a dejar un 10% de SBS (sampling_strategy = 0.1) y 90% NoSBS, para luego, hacer oversamplig de SBS...
  #undersample = RandomUnderSampler(random_state=_random_state, sampling_strategy = 0.1)

  # vamos a dejar igual peso, 50% de SBS, pporque vamos a repetir esto muchas veces
  undersample = RandomUnderSampler(random_state=_random_state, sampling_strategy = 1)
  #oversample = SMOTE(random_state=random_state)

  # Separamos en X e y
  X_SBS = df.drop(['52'], axis=1)
  y_SBS = df['52']

  #hacemos el submuestreo
  X_over_SBS, y_over_SBS = undersample.fit_resample(X_SBS , y_SBS)


  # preparamos todo para agrupar despues
  df = X_over_SBS
  df['52'] = y_over_SBS
  dfSBS2 = df[(df['52'] == 1)]
  dfNoSBS2 = df[(df['52'] == 0)]
  dfSBS2.drop(['52'], axis=1,inplace= True)
  dfNoSBS2.drop(['52'], axis=1,inplace= True)

  return dfSBS2, dfNoSBS2

#print(len(dfSBS2), len(dfNoSBS2), len(X_over_SBS), len(df))

# Agrupador prepM3b  (y submuestreo del modelo agrupado)

def desvio0(min, max, desvio)->float:
  ret = float('nan')
  if not math.isnan(min):
    ret = desvio
    if min == max:
      ret = 0.0
  return ret


#preprocesamiento y union de los dataset. Variable target: SBS sii df['52']==1
def prepM3b(df, clase):

  #2: obsid, 4: fuente   mantenemos la fuente, para poder hacer el ranking
#   df = df.groupby(['4'],group_keys=True).agg(
  df = df.groupby(['4'],group_keys=True).agg(
      #count=('2','count'),  # cant obs

      SBS=('53', 'min'),
      min110=('110', 'min'),
      max110=('110', 'max'),
      mean110=('110', 'mean'),
      std110=('110', 'std'),

      min111=('111', 'min'),
      max111=('111', 'max'),
      mean111=('111', 'mean'),
      std111=('111', 'std'))

  #corregimos el desvio muestral (cuando hay 1 sola muestra)
  df['std110'] = df.apply(lambda x: desvio0(x['min110'],x['max110'],x['std110']), axis=1)
  df['std111'] = df.apply(lambda x: desvio0(x['min111'],x['max111'],x['std111']), axis=1)

  #variable target
  df['52'] = clase

  return df


def prep():
  dfSBS = prepM3(ruta + 'Swift05_11SBS165.csv', 1, ruta + 'Swift05_11SBS52.csv')
  dfNoSBS = prepM3(ruta + 'Swift11NoSBS.csv', 0)

  #imputacion por regresion lineal (en toda la base)
  regresionLineal(dfSBS)
  regresionLineal(dfNoSBS)

  frames = [dfSBS, dfNoSBS]
  return pd.concat(frames)



In [ ]:
#------------------------------------------------ split y entrenamiento
#  _random_state para sub muestrear diferente que el entrenamiento (random_state, global, es para entrenar)

from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree

sns.set(context='notebook')

#  _random_state para muestrear diferente que el entrenamiento (random_state <-- global)
def RF(df, _random_state, _class_weight='balanced'):

  dfSBS2, dfNoSBS2 = subMuestrear(df, _random_state)

  #agrupamiento
  dfSBS2 = prepM3b(dfSBS2,1)
  dfNoSBS2 = prepM3b(dfNoSBS2,0)

  #print('dfSBS2',len(dfSBS2), 'dfNoSBS2', len(dfNoSBS2))

  frames = [dfSBS2, dfNoSBS2]
  df2 = pd.concat(frames)
  # Separamos en X e y. Quitamos 'SBS'
  X_SBS = df2.drop(['52','SBS'], axis=1)
  y_SBS = df2['52']

  # 30% para test y 70% para train
  X_train_SBS, X_test_SBS, y_train_SBS, y_test_SBS = train_test_split(X_SBS,y_SBS, test_size=0.30,random_state=_random_state)

  print('X_train_SBS',len(X_train_SBS), 'X_test_SBS', len(X_test_SBS))

  #notar que random_state es global y fijo para todo el experimiento
  clf = RandomForestClassifier(random_state=random_state, class_weight= _class_weight)

  param_grid = {'n_estimators':[10,20,30], 'criterion': ['gini', 'entropy'], 'max_depth': [3,4,5,6,7,8], 'min_samples_leaf' : [3]} #, 'min_samples_leaf' : [5]}

  # Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
  grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, return_train_score=True)

  grid_search.fit(X_train_SBS, y_train_SBS)

  return grid_search, X_train_SBS, X_test_SBS, y_test_SBS, dfSBS2


#Metricas y Graficamos matriz de confusion
from sklearn.metrics import accuracy_score

def imprimirMetricas(mcTexto, mcGrafico, redondeo, y_train_SBS, X_test_SBS, y_test_SBS,y_test_pred_SBS, best_clf, soloF1, pesoSBS = None):

  cm = confusion_matrix(y_test_SBS,y_test_pred_SBS,labels=best_clf.classes_)
  TN, FP, FN, TP = cm.ravel()
  #  (con las clases 1,0)
  if mcGrafico:
    ConfusionMatrixDisplay(cm, display_labels=np.flip(best_clf.classes_)).plot()

  # Calculo de las predicciones en Train y test
  y_train_pred = best_clf.predict(X_train_SBS)
  y_test_pred = best_clf.predict(X_test_SBS)

  if not soloF1:
    #print('El accuracy en train es:',round(accuracy_score(y_train_SBS,y_train_pred_SBS),redondeo))
    print('El accuracy en test es:', round(accuracy_score(y_test_SBS,y_test_pred_SBS),redondeo))

  #print(classification_report(y_test_SBS,y_test_pred_SBS))
  accu = (TP + TN) / (TP + FP + FN + TN)
  pres = TP / (TP + FP)
  rec = TP / (TP + FN)  # TP / (FN + TP)
  f1 = 2 * pres * rec / (pres + rec)

  if soloF1:
    if pesoSBS != None:
      print("pesoSBS", round(pesoSBS, redondeo), "F1", round(f1,redondeo))
    else:
      print( "F1", round(f1,redondeo))
  else:
    if mcTexto:
      print("Matriz de confusión")
      print("Predicho (1) (0)")
      print("Real(1) ",TP,"  ", FN)
      print("Real(0) ",FP,"  ", TN)

    print("Accuracy :  ", round(accu,redondeo))
    print("Precision : ", round(pres,redondeo))
    print("Recall :    ", round(rec,redondeo))
    print("F1 :        ", round(f1,redondeo))


def agregarSBSNombres(X_test_SBS, dfSBS2, y_test_pred_SBS):
#le agregamos los nombres de las SBS y la prediccion
  X_test_SBS = X_test_SBS.merge(dfSBS2, how='left', on='4')

  X_test_SBS.drop(['min110_y','min111_y','max110_y','max111_y','mean110_y','mean111_y','std110_y','std111_y'], axis=1, inplace=True)

  X_test_SBS['pred'] = y_test_pred_SBS

  soloSBS = X_test_SBS[X_test_SBS['SBS'].notna()]

  return soloSBS


In [ ]:
#------------empezar aca


df = prep()
pesoSBS = 0.91 #peso optimo surge de experimento anterior para el caso desbalceado solamente


/tmp/ipython-input-2825026973.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(np.int64)
/tmp/ipython-input-2825026973.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(np.int64)


In [ ]:

# split y entrenamiento. en cada ciclo
# por ahora "igual peso". sino poner:  {1:pesoSBS, 0: 1-pesoSBS}


cantCorridas = 20 #se toman semillas (random_state) de 0 a cantCorridas
resultados = np.empty(cantCorridas, dtype = pd.DataFrame)

i=0
for random_state_i in range(cantCorridas):
  grid_search, X_train_SBS, X_test_SBS, y_test_SBS, dfSBS2 = RF(df, random_state_i)

    # Modelo decision tree con parametros optimizados
  best_clf = grid_search.best_estimator_

    # Predecimos Y
  y_train_pred_SBS = best_clf.predict(X_train_SBS)
  y_test_pred_SBS = best_clf.predict(X_test_SBS)

  #xxx opcional
  #imprimirMetricas(True, False, 4,X_train_SBS, X_test_SBS, y_test_SBS, y_test_pred_SBS, best_clf, False)

  #me guardo la prediccion de todas las SBS en cada iteracion
  resultados[i]= agregarSBSNombres(X_test_SBS, dfSBS2, y_test_pred_SBS)
  print("dataframe ", i)
  i = i +1



In [ ]:
#estadisticas

dic = {}

#recorro todos los dataframes con las clasificaciones de SBS
for i in range(len(resultados)):

  for index, row in resultados[i].iterrows():
    if int(row['52'])== int(row['pred']):
      if row['SBS'] in dic:
        dic[row['SBS']] = (dic[row['SBS']][0]+1,dic[row['SBS']][1])
      else:
        dic[row['SBS']] = (1,0)
    else:
      if row['SBS'] in dic:
        dic[row['SBS']] = (dic[row['SBS']][0],dic[row['SBS']][1]+1)
      else:
        dic[row['SBS']] = (0,1)

#for clave in dic.keys():
#  print(clave, dic[clave])



EM* AS  453 (6, 0)
V* AX Per (4, 0)
M33SyS J013239.11+303836.5 (0, 6)
V* RW Hya (3, 0)
EM* AS  297 (5, 0)
V* ZZ CMi (9, 0)
V* ER Del (6, 0)
V* V420 Hya (8, 0)
V* CN Cha (3, 2)
M33SyS J013334.95+305108.3 (0, 6)
OGLE BLG-LPV-26902 (0, 9)
V* SY Mus (7, 0)
M31SyS J004335.01+414358.2 (0, 7)
NAME UKS-Ce1 (2, 2)
V* RS Oph (7, 0)
EM* MWC  603 (8, 0)
V* AR Pav (6, 0)
V* KX TrA (7, 0)
V* RT Cru (5, 0)
EM* AS  289 (4, 0)
CD-28  3719 (9, 0)
Haro 1-2 (6, 0)
M31SyS J004117.11+404524.1 (0, 7)
GSC 06806-00016 (5, 1)
V* Z And (9, 0)
V* CH Cyg (8, 0)
V* TX CVn (8, 0)
WRAY 15-1511 (9, 1)
BD-21  3873 (7, 0)
V* UV Aur (7, 0)
EM* StHA  169 (6, 0)
V* NQ Gem (8, 0)
WRAY 16-148 (6, 1)
V* T CrB (7, 0)
M33SyS J013303.27+303528.3 (1, 10)
LGGS J004216.72+404415.8 (0, 5)
NAME SMC Symbiotic star 2 (10, 1)
WRAY 15-619 (2, 4)
EM* StHA   32 (9, 0)
EM* StHA  190 (5, 0)
V* LT Del (1, 0)
M31SyS J004155.61+410846.7 (5, 1)
WRAY 16-208 (7, 2)
V* CI Cyg (3, 0)
Hen 3-863 (6, 0)
V* BI Cru (3, 1)
V* V934 Her (5, 1)
V* Y CrA (4, 

In [ ]:
import csv

def dict_of_lists_to_csv(dictionary, filename):
    keys = list(dictionary.keys())
    values = list(dictionary.values())

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(keys) # Write the column headers
        writer.writerows(zip(*values)) # Transpose and write the values

dict_of_lists_to_csv(dic, 'rankingSBS.csv')

In [ ]:
#le agregamos los nombres de las SBS y la prediccion
X_test_SBS = X_test_SBS.merge(dfSBS2, how='left', on='4')

X_test_SBS.drop(['min110_y','min111_y','max110_y','max111_y','mean110_y','mean111_y','std110_y','std111_y'], axis=1, inplace=True)

X_test_SBS['pred'] = y_test_pred_SBS

soloSBS = X_test_SBS[X_test_SBS['SBS'].notna()]


In [ ]:
soloSBS.to_csv("SBSPredic.csv", sep=';', encoding='utf-8',index=False)

In [ ]:
#ojo, que guarda sin fuentes!
dfSBS.to_csv("Swift05_11SBS_Agrupado.csv", sep=';', encoding='utf-8',index=False)
dfNoSBS.to_csv("Swift05_11NoSBS_Agrupado.csv", sep=';', encoding='utf-8',index=False)